In [1]:
import pandas as pd
from sklearn.metrics import make_scorer, balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from hyperopt import hp
import numpy as np
from hyperopt import Trials, tpe, fmin
import warnings
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from nbimporter import NotebookLoader

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("../code/output.csv",index_col=False)

In [3]:
rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)

In [4]:
selected_columns=["locus tag","essential","DNA","protein sequence"]
Xs= df.drop(columns=selected_columns)
y=df["essential"]

In [5]:
from sklearn.linear_model import LassoCV

lassoCV = LassoCV(cv=20,random_state=10)
all_feature_names = ["GC_Content","CAI","A","R","N","D","C","Q","E","G","H","I","L","K","M","F","P","S","T","W","Y","V","nSE2","nSE3","nGE2","nGE3"]

In [7]:
space = {
    'n_estimators': hp.quniform('n_estimators', 100, 1000, 100),  # 树的数量
    'max_depth': hp.quniform('max_depth', 3, 10, 1),  # 每棵树的最大深度
    'learning_rate': hp.loguniform('learning_rate', -3, 0),  # 学习率
    'subsample': hp.uniform('subsample', 0.5, 1.0),  # 每棵树的样本采样比例
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),  # 每棵树的特征采样比例
    'gamma': hp.loguniform('gamma', -3, 2),  # 节点分裂时损失函数减小值的最小值要求
    'reg_alpha': hp.loguniform('reg_alpha', -3, 2),  # L1 正则化项系数
    'reg_lambda': hp.loguniform('reg_lambda', -3, 2),  # L2 正则化项系数
}

In [8]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


def xgb_ac_cv(params):
    scorer = make_scorer(accuracy_score)
    model = XGBClassifier(
        n_estimators=int(params['n_estimators']),
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        objective='binary:logistic'  # 二分类问题
    )
    scorer = make_scorer(accuracy_score)
    score = -cross_val_score(model, X_resampled, y_resampled, cv=5,scoring=scorer).mean()
    return score

In [9]:
from sklearn.model_selection import KFold
from joblib import load
from sklearn.feature_selection import RFE

scores = []
score = 0
models = []
model_xgb = load("../../model/XGB_model.joblib")
kf = KFold(n_splits=10, shuffle=True, random_state=42)
features_xgb = []
for train_index, test_index in kf.split(Xs):
    X_train_fold, Xs_test = Xs.iloc[train_index], Xs.iloc[test_index]
    y_train_fold, y_test = y.iloc[train_index], y.iloc[test_index]
    X_resampled, y_resampled = rus.fit_resample(X_train_fold, y_train_fold)
    rfe = RFE(model_xgb, n_features_to_select=20)
    rfe.fit(X_resampled, y_resampled)
    X_resampled = rfe.transform(X_resampled)  # For training data
    Xs_test = rfe.transform(Xs_test)  # For testing data
    lassoCV.fit(X_resampled,y_resampled)
    lassoCV.fit(Xs_test,y_test)
    selected_features_rfe = [all_feature_names[i] for i in range(len(all_feature_names)) if rfe.support_[i]]
    features= [selected_features_rfe[i] for i, coef in enumerate(lassoCV.coef_) if coef != 0]
    print(features)
    trials = Trials()
    best=fmin(fn=xgb_ac_cv, # function to optimize
              space=space, 
              algo=tpe.suggest, # optimization algorithm, hyperotp will select its parameters automatically
              max_evals=50, # maximum number of iterations
              trials=trials, # logging
              rstate=np.random.default_rng(42) # fixing random state for the reproducibility
    )
    
    model = XGBClassifier(
        n_estimators=int(best['n_estimators']),
        max_depth=int(best['max_depth']),
        learning_rate=best['learning_rate'],
        subsample=best['subsample'],
        colsample_bytree=best['colsample_bytree'],
        gamma=best['gamma'],
        reg_alpha=best['reg_alpha'],
        reg_lambda=best['reg_lambda'],
        objective='binary:logistic'  # 二分类问题
    )
    models.append(model)
    model.fit(X_resampled,y_resampled)
    tpe_test_score=accuracy_score(y_test, model.predict(Xs_test))
    scores.append(tpe_test_score)
    print(tpe_test_score)
    if score < tpe_test_score:
        score = tpe_test_score
        best_model = model
        features_xgb = features

['CAI', 'D', 'E', 'H', 'I', 'F', 'P', 'T', 'W', 'V', 'nSE3']
100%|██████████| 50/50 [00:50<00:00,  1.01s/trial, best loss: -0.6577717879604672]
0.6304347826086957
['GC_Content', 'CAI', 'E', 'S']
100%|██████████| 50/50 [01:18<00:00,  1.58s/trial, best loss: -0.6209664209664209]
0.6521739130434783
['GC_Content', 'CAI', 'E', 'L', 'nSE3']
100%|██████████| 50/50 [01:12<00:00,  1.45s/trial, best loss: -0.6150943396226415]
0.6521739130434783
['CAI']
100%|██████████| 50/50 [01:13<00:00,  1.47s/trial, best loss: -0.6140341419586703]
0.6473429951690821
['CAI', 'F', 'nGE2']
100%|██████████| 50/50 [01:00<00:00,  1.22s/trial, best loss: -0.658688127379716] 
0.6256038647342995
['GC_Content', 'CAI', 'D', 'C', 'Q', 'H', 'L', 'F', 'V', 'nSE2']
100%|██████████| 50/50 [01:11<00:00,  1.42s/trial, best loss: -0.6226371826371826]
0.6763285024154589
['CAI', 'nSE3']
100%|██████████| 50/50 [01:19<00:00,  1.58s/trial, best loss: -0.6428304705003735]
0.6473429951690821
['GC_Content', 'A', 'R', 'D', 'E', 'G', 'H'

In [10]:
print(score)

0.6763285024154589


In [12]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score

scores_xgb_ac = []
scores_xgb_mcc = []
scores_xgb_auc = []
scores_xgb_f1 = []
scores_xgb_precision = []
scores_xgb_recall = []
for train_index, test_index in kf.split(Xs):
    X_train_fold, Xs_test = Xs.iloc[train_index], Xs.iloc[test_index]
    y_train_fold, y_test = y.iloc[train_index], y.iloc[test_index]
    X_resampled, y_resampled = rus.fit_resample(X_train_fold, y_train_fold)
    best_model.fit(X_resampled[features_xgb],y_resampled)
    y_predict = best_model.predict(Xs_test[features_xgb])
    acc=accuracy_score(y_test, y_predict)
    scores_xgb_ac.append(acc)
    auc = roc_auc_score(y_test,y_predict)
    scores_xgb_auc.append(auc)
    f1 = f1_score(y_test,y_predict)
    scores_xgb_f1.append(f1)
    precision = precision_score(y_test,y_predict)
    scores_xgb_precision.append(precision)
    recall = recall_score(y_test,y_predict)
    scores_xgb_recall.append(recall)

In [14]:
def generateString(scores):
    mean_score = round(np.mean(scores),4)
    var_score = round(np.var(scores),4)
    return f'{mean_score}±{var_score}'


In [15]:
from joblib import dump

dump(best_model, '../../model/XGB_model_Ecoli.joblib')

['../../model/XGB_model_Ecoli.joblib']

In [16]:
scores2 = []
score = 0
models = []
kf = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kf.split(Xs):
    X_train_fold, Xs_test = Xs.iloc[train_index], Xs.iloc[test_index]
    y_train_fold, y_test = y.iloc[train_index], y.iloc[test_index]
    X_resampled, y_resampled = rus.fit_resample(X_train_fold, y_train_fold)
    trials = Trials()
    best=fmin(fn=xgb_ac_cv, # function to optimize
              space=space, 
              algo=tpe.suggest, # optimization algorithm, hyperotp will select its parameters automatically
              max_evals=50, # maximum number of iterations
              trials=trials, # logging
              rstate=np.random.default_rng(42) # fixing random state for the reproducibility
    )
    
    model = XGBClassifier(
        n_estimators=int(best['n_estimators']),
        max_depth=int(best['max_depth']),
        learning_rate=best['learning_rate'],
        subsample=best['subsample'],
        colsample_bytree=best['colsample_bytree'],
        gamma=best['gamma'],
        reg_alpha=best['reg_alpha'],
        reg_lambda=best['reg_lambda'],
        objective='binary:logistic'
    )
    models.append(model)
    model.fit(X_resampled,y_resampled)
    tpe_test_score=accuracy_score(y_test, model.predict(Xs_test))
    scores2.append(tpe_test_score)
    print(tpe_test_score)
    if score < tpe_test_score:
        score = tpe_test_score
        best_model2 = model

100%|██████████| 50/50 [01:42<00:00,  2.05s/trial, best loss: -0.64062893081761]  
0.6014492753623188
100%|██████████| 50/50 [01:25<00:00,  1.70s/trial, best loss: -0.6172645372645371]
0.6231884057971014
100%|██████████| 50/50 [01:42<00:00,  2.06s/trial, best loss: -0.5792452830188679]
0.6763285024154589
100%|██████████| 50/50 [01:37<00:00,  1.94s/trial, best loss: -0.6196406109613657]
0.5966183574879227
100%|██████████| 50/50 [01:31<00:00,  1.82s/trial, best loss: -0.6623399100034615]
0.6135265700483091
100%|██████████| 50/50 [01:38<00:00,  1.97s/trial, best loss: -0.6172972972972973]
0.6980676328502415
100%|██████████| 50/50 [01:23<00:00,  1.68s/trial, best loss: -0.6335698282300224]
0.6135265700483091
100%|██████████| 50/50 [02:41<00:00,  3.23s/trial, best loss: -0.6346271338724169]
0.5990338164251208
100%|██████████| 50/50 [02:19<00:00,  2.78s/trial, best loss: -0.6382210242587602]
0.6642512077294686
100%|██████████| 50/50 [01:36<00:00,  1.93s/trial, best loss: -0.6193171608265947]

In [20]:
scores_xgb_ac2 = []
scores_xgb_auc2 = []
scores_xgb_f12 = []
scores_xgb_precision2 = []
scores_xgb_recall2 = []
for train_index, test_index in kf.split(Xs):
    X_train_fold, Xs_test = Xs.iloc[train_index], Xs.iloc[test_index]
    y_train_fold, y_test = y.iloc[train_index], y.iloc[test_index]
    X_resampled, y_resampled = rus.fit_resample(X_train_fold, y_train_fold)
    best_model2.fit(X_resampled,y_resampled)
    y_predict = best_model2.predict(Xs_test)
    acc=accuracy_score(y_test, y_predict)
    scores_xgb_ac2.append(acc)
    auc = roc_auc_score(y_test,y_predict)
    scores_xgb_auc2.append(auc)
    f1 = f1_score(y_test,y_predict)
    scores_xgb_f12.append(f1)
    precision = precision_score(y_test,y_predict)
    scores_xgb_precision2.append(precision)
    recall = recall_score(y_test,y_predict)
    scores_xgb_recall2.append(recall)

In [21]:
import csv
with open('XGB.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Type','ACC','AUC','F1','Precision', 'Recall'])
    writer.writerow(['XGB',generateString(scores_xgb_ac),generateString(scores_xgb_auc),generateString(scores_xgb_f1),generateString(scores_xgb_precision),generateString(scores_xgb_recall)])
    writer.writerow(['XGB',generateString(scores_xgb_ac2),generateString(scores_xgb_auc2),generateString(scores_xgb_f12),generateString(scores_xgb_precision2),generateString(scores_xgb_recall2)])

In [22]:
print(features_xgb)

['GC_Content', 'CAI', 'D', 'C', 'Q', 'H', 'L', 'F', 'V', 'nSE2']


In [23]:
from scipy.stats import ks_2samp
p_values = []
for train_index, test_index in kf.split(Xs):
    X_train_fold, Xs_test = Xs.iloc[train_index], Xs.iloc[test_index]
    y_train_fold, y_test = y.iloc[train_index], y.iloc[test_index]
    X_resampled, y_resampled = rus.fit_resample(X_train_fold, y_train_fold)
    best_model.fit(X_resampled[features_xgb],y_resampled)
    y_predict_fs = best_model.predict(Xs_test[features_xgb])
    best_model2.fit(X_resampled,y_resampled)
    y_predict = best_model2.predict(Xs_test)
    ks_statistic, p_value = ks_2samp(y_predict_fs,y_predict)
    p_values.append(p_value)
print(np.mean(p_values))

0.6621545224907361
